# Mapping data
This is an alternative to using the google maps api, it requires no API key and no credit card.  The documentation for installing is here:  https://ipyleaflet.readthedocs.io/en/latest/installation.html




In [67]:
from ipyleaflet import Map, Marker, Icon, CircleMarker, Heatmap

center = (52.204793, 360.121558)

m = Map(center=center, zoom=15)

marker = Marker(location=center, draggable=False)
m.add_layer(marker);

m

Map(center=[52.204793, 360.121558], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

for some reason the default marker appears broken at the moment.  See below for a couple of work arounds.

In [44]:
import pandas as pd

stations = pd.read_csv("../Data/bikeshare_stations.csv")


In [45]:
locations = list(zip(stations.latitude, stations.longitude))
dc_center = (38.9072, -77.0369)
dcmap = Map(center=dc_center, zoom=12)

In [46]:
icon = Icon(icon_url="https://runestone.academy/runestone/static/marker.png")
for loc in locations:
    marker = CircleMarker(location=loc, radius=2)
    #marker= Marker(location=loc, draggable=False, icon=icon)
    dcmap.add_layer(marker)
    
dcmap

Map(center=[38.9072, -77.0369], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [47]:
stations.head()

,station_id,name,status,latitude,longitude
0,31620,5th & F St NW,open,38.896301,-77.027463
1,31105,14th & Harvard St NW,open,38.926571,-77.032414
2,31400,Georgia & New Hampshire Ave NW,open,39.221500,-77.059858
3,31111,10th & U St NW,open,38.916804,-77.026235
4,31104,Adams Mill & Columbia Rd NW,open,38.922680,-77.042667


## Make a Heatmap

Lets make a heatmap based on the number of trips originating at each stations.  Good review of merging and grouping by data with the bonus of getting see it on a map.

It might also be interesting to figure out the count of trips between each station and then use the midpoint as the center for each location.

In [48]:
trip_data = pd.read_csv('../Data/trip_data.csv')
trip_data.head()

,duration,start_date,end_date,start_station,end_station,bike_number,member_type
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,W00247,Member
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,W00675,Casual
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,W00357,Member
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,W00970,Member
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,W00346,Casual


In [60]:
tc = trip_data.groupby('start_station').agg(trip_count=('end_station','count'))



In [62]:
tc = tc.reset_index()

In [64]:
hm = tc.merge(stations, left_on='start_station', right_on='station_id')

In [70]:
locations = list(zip(hm.latitude, hm.longitude, hm.trip_count))

In [73]:
m = Map(center=dc_center, zoom=12)

heatmap = Heatmap(
    locations=locations,
    radius=10
)

m.add_layer(heatmap);

m

Map(center=[38.9072, -77.0369], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…